In [ ]:
!pip install websocket_client

In [ ]:
from websocket import WebSocketApp
import time
import random
import json
import _thread as thread



In [ ]:
# url = 'wss://aispawn.herokuapp.com/ws'
url = "ws://localhost:3000/ws"

In [ ]:
def line_intersection(line1, line2):
    xdiff = (line1[0][0] - line1[1][0], line2[0][0] - line2[1][0])
    ydiff = (line1[0][1] - line1[1][1], line2[0][1] - line2[1][1])

    def det(a, b):
        return a[0] * b[1] - a[1] * b[0]

    div = det(xdiff, ydiff)
    if div == 0:
       raise Exception('lines do not intersect')

    d = (det(*line1), det(*line2))
    x = det(d, xdiff) / div
    y = det(d, ydiff) / div
    return x, y


In [ ]:

def get_interssection(d):
  bullet_speed = 15
  player_speed = 5

  if d['vel_x'] == 0:
    player_slope = 9999
    bot_slope = 0
  else:
    player_slope = d['vel_y']/d['vel_x']
    if d['vel_y']==0:
      bot_slope=9999
    else:
      bot_slope = -1/player_slope

  A = (d['px'], d['py'])
  B=(A[0] + 1, A[1] + player_slope)
  C = (d['bx'], d['by'])
  D = (C[0]+1, C[1]+bot_slope)
  poi = line_intersection((A, B),(C,D))


In [ ]:
def ws_handler(ws, message):

  msg=json.loads(message)
  if msg['type']=='sync_player':

      this_player = msg['you_are']
      other = 1-this_player
      players = msg['info'][0]
      px = players[other]['x']
      py = players[other]['y']

      bx=players[this_player]['x']
      by=players[this_player]['y']
      ws.send(f'fire{bx-px}, {by-py}')
  

In [ ]:

def on_error(ws, error):
    print(error)


In [ ]:
def on_open(ws):
    def run(*args):
      while True:
        time.sleep(0.4)
        ws.send('sync')
    ws.send('nameBotBoio')
    thread.start_new_thread(run, ())



In [ ]:
ws = WebSocketApp(url, on_message=ws_handler, on_error = on_error)
ws.on_open = on_open
ws.run_forever()

False

In [ ]:
ws.close()